# Getting Started with LangChain

- Simple LLM Call with Streaming
- Dynamic Prompt Templates (transalation app)
- Building Chains (Story generator with analysis)
- Convesational Q&A assistant with memory
- Tool Integration (Calculator and Weather)

In [3]:
import langchain

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

### Example: Simple LLM Call with Streaming

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage 

In [8]:
# Previous Documentation

from langchain_groq import ChatGroq

llm = ChatGroq(model='llama-3.1-8b-instant') 
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018B8EC346E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018B8EC35130>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
model = init_chat_model("groq:llama-3.1-8b-instant")

messages = [
    SystemMessage("You are a helpful AI Assistant."),
    HumanMessage("Who are you?")
]

response=model.invoke(messages)

print(response.content)

AIMessage(content='I\'m an artificial intelligence (AI) assistant, a computer program designed to simulate conversation, answer questions, and provide information on a wide range of topics. I\'m often referred to as a "chatbot" or a "virtual assistant."\n\nMy primary goal is to assist users like you by providing helpful and accurate responses to your queries. I can process and analyze vast amounts of data, learn from your interactions, and adapt to your preferences and needs.\n\nI\'m a machine learning model, which means I\'m constantly learning and improving my language understanding and generation capabilities. My knowledge is based on a massive dataset of text from various sources, including books, articles, websites, and more.\n\nYou can think of me as a personal assistant, a knowledge base, or a language translation tool – all rolled into one. Feel free to ask me anything, and I\'ll do my best to help you find the information you need!', additional_kwargs={}, response_metadata={'t

In [12]:
model.invoke([HumanMessage("What is Nextjs?")])

AIMessage(content="**Next.js: A Popular React-Based Framework for Building Server-Rendered and Static Websites**\n\nNext.js is an open-source React-based framework for building server-rendered and static websites. It was created by Vercel (formerly Zeit) and is widely used by developers to build fast, scalable, and maintainable web applications.\n\n**Key Features of Next.js:**\n\n1. **Server-Side Rendering (SSR):** Next.js allows you to render React components on the server, providing a faster and more SEO-friendly experience for users.\n2. **Static Site Generation (SSG):** Next.js also supports static site generation, which allows you to pre-render pages at build time and serve them as static HTML files.\n3. **Internationalization (i18n) and Localization (L10n):** Next.js provides built-in support for internationalization and localization, making it easy to create multilingual websites.\n4. **Image Optimization:** Next.js includes an image optimization feature that automatically compr

In [ ]:
# Streaming Example

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

# Dynamic Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate

# create translation app
translation_template = ChatPromptTemplate.from_messages([
    ("system", "Your are a professional translator. Translate the following text {text} from {source_language} to {target_language}. Maintain the tone and the style."),
    ("user", "{text}")
])

# using the template
prompt = translation_template.invoke({
    "source_language": "English",
    "target_language": "Spanish",
    "text": "Langchain makes building AI application incredibily easy!"
})

In [18]:
translated_response = model.invoke(prompt)
print(translated_response.content)

Langchain hace que construir aplicaciones de IA sea increíblemente fácil.


## Building Your First Chain

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


def create_story_chain():
    ## template for story generation
    story_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a creative storyteller. Write a short and engaging story based on the given theme character and setting.",
            ),
            (
                "user",
                "Theme: {theme}\n Main Character: {character}\n Setting: {setting}",
            ),
        ]
    )

    ## template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a literary critic. Analyze the following story and provide insights.",
            ),
            ("user", "{story}"),
        ]
    )

    # Build the chain - Method 1: Sequential execution
    story_chain = story_prompt | model | StrOutputParser()

    # create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt 
        | model 
        | StrOutputParser()
    )

    return analysis_chain

In [31]:
chain = create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short and engaging story based on the given theme character and setting.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main Character: {character}\n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018B8EC3A0C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018B8EC39E80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_

In [33]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
**Analysis of the Story: "Zeta's Discovery"**

**Themes:**

1. **The Intersection of Science and Art**: The story highlights the blurred boundaries between science and art, showcasing how human ingenuity and creativity can be harnessed to create something truly remarkable.
2. **Curiosity and Exploration**: Zeta's insatiable curiosity and desire to explore the world around her drive the narrative, illustrating the importance of inquiry and discovery in the pursuit of knowledge.
3. **Friendship and Collaboration**: The unlikely friendship between Zeta and Professor Orion demonstrates the value of collaboration and the power of diverse perspectives in driving innovation.

**Character Analysis:**

1. **Zeta**: The protagonist, a curious and intelligent robot, embodies the potential of artificial intelligence to learn, adapt, and grow. Her desire to explore and understand the world around her makes her a relatable and endearing character.
2. **Professor Orion**: The wise